In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
tqdm.tqdm.pandas()

In [2]:
date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')

In [3]:
date_today_record = str(date_today).split(' ')[0].replace('-', '')
date_today_record

'20220225'

In [4]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [5]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [6]:
def data_prep(bdtype, tradetype, starting_year=2018):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    if (tradetype != '매매') & (tradetype != '전월세'):
        raise ValueError('두번째 변수는 매매 또는 전월세만 입력 가능')
    
    
    basedir = './국토교통부_실거래가_공개시스템/강원도/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    dfs_list = []
    for i, f in tqdm.tqdm_notebook(enumerate(filenames)):
        splitted_filename = f.split('실거래가_')       
        file_year = int(splitted_filename[-1][:4])
        
        '''if file_year < starting_year:
            continue'''
        
        try:
            df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        except:
            try:
                df = pd.read_csv(basedir + f, encoding='utf-8', header=15)
            except:
                df = pd.read_csv(basedir + f, header=15)
        
        if '건물명' in df.columns:
            df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
        elif '단지명' in df.columns:
            df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
                
            
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
            
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    if tradetype == '전월세':
        concat_df = concat_df.rename(columns={'전월세구분':'거래구분', '보증금(만원)':'거래금액(만원)'})
        concat_df = concat_df.drop(columns=['월세(만원)'])
        concat_df = concat_df[concat_df['거래구분'] == '전세']
    elif tradetype == '매매':
        concat_df['거래구분'] = '매매'
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].astype('Int64')
    concat_df['계약일'] = concat_df['계약일'].astype('Int64')
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['번지', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [7]:
apart_df = data_prep('아파트', '매매')
apart_df['부동산유형'] = '아파트'
print(apart_df.shape)
apart_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(80191, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,강원도 강릉시 견소동 202-0,경강로2539번길 8,강원도 강릉시 견소동,202,0,송정한신,84.945,17500,10,1997.0,-,-,매매,21.0,2018-01-03,206.015657,25.0,아파트
1,강원도 강릉시 견소동 202-0,경강로2539번길 8,강원도 강릉시 견소동,202,0,송정한신,59.800,12500,13,1997.0,-,-,매매,21.0,2018-02-20,209.030100,25.0,아파트
2,강원도 강릉시 견소동 202-0,경강로2539번길 8,강원도 강릉시 견소동,202,0,송정한신,43.380,9800,4,1997.0,-,-,매매,21.0,2018-03-08,225.910558,25.0,아파트
3,강원도 강릉시 견소동 202-0,경강로2539번길 8,강원도 강릉시 견소동,202,0,송정한신,84.945,17500,10,1997.0,-,-,매매,21.0,2018-03-20,206.015657,25.0,아파트
4,강원도 강릉시 견소동 202-0,경강로2539번길 8,강원도 강릉시 견소동,202,0,송정한신,39.080,10800,10,1997.0,-,-,매매,21.0,2018-03-25,276.356192,25.0,아파트


In [8]:
yunrip_df = data_prep('연립다세대', '매매')
yunrip_df['부동산유형'] = '연립다세대'
print(yunrip_df.shape)
yunrip_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(7218, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,강원도 강릉시 교동 203-10,율곡로2874번길 19,강원도 강릉시 교동,203,10,203-10,80.4600,7900,1,1980,-,-,매매,38,2018-08-22,98.185434,42,연립다세대
1,강원도 강릉시 교동 994-64,강릉대로121번길 38,강원도 강릉시 교동,994,64,교동대호,84.9600,11000,2,2001,-,-,매매,17,2018-04-18,129.472693,21,연립다세대
2,강원도 강릉시 교동 941-9,원대로8번길 42,강원도 강릉시 교동,941,9,교동맨션,84.2400,8200,2,1984,-,-,매매,34,2018-03-26,97.340931,38,연립다세대
3,강원도 강릉시 교동 302-0,화부산로141번안길 3,강원도 강릉시 교동,302,0,까르빌,67.9961,18600,4,2018,-,-,매매,0,2018-12-24,273.545100,4,연립다세대
4,강원도 강릉시 교동 302-0,화부산로141번안길 3,강원도 강릉시 교동,302,0,까르빌,73.2114,18800,3,2018,-,-,매매,0,2018-12-26,256.790609,4,연립다세대


In [9]:
officetel_df = data_prep('오피스텔', '매매')
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

(1991, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,강원도 강릉시 연곡면 영진리 357-30,영진길 33,강원도 강릉시 연곡면 영진리,357,30,ES프라우드,66.41,13143,3,2016.0,-,-,매매,2.0,2018-04-04,197.906942,6.0,오피스텔
1,강원도 강릉시 연곡면 영진리 357-30,영진길 33,강원도 강릉시 연곡면 영진리,357,30,ES프라우드,65.63,21844,7,2016.0,-,-,매매,2.0,2018-08-11,332.835593,6.0,오피스텔
2,강원도 강릉시 연곡면 영진리 357-30,영진길 33,강원도 강릉시 연곡면 영진리,357,30,ES프라우드,63.07,11987,2,2016.0,-,-,매매,2.0,2018-08-24,190.058665,6.0,오피스텔
3,강원도 강릉시 유천동 776-4,사임당로 180,강원도 강릉시 유천동,776,4,유천올림피아오피스텔,21.66,7940,4,2016.0,-,-,매매,2.0,2018-07-14,366.574331,6.0,오피스텔
4,강원도 강릉시 홍제동 1018-2,선수촌로 79-29,강원도 강릉시 홍제동,1018,2,골드팰리스,18.66,7100,6,2016.0,-,-,매매,2.0,2018-01-10,380.493033,6.0,오피스텔


In [10]:
house_df = pd.concat([apart_df, yunrip_df, officetel_df]).sort_values(['시군구', '본번', '부번', '건축년도', '전용면적(㎡)', '계약날짜']).reset_index(drop=True)
house_df = house_df.drop(columns=['거래유형', '중개사소재지', '거래구분', '계약날짜기준_건물연식'])
del apart_df, yunrip_df, officetel_df
print(house_df.shape)
house_df.head()

(89400, 14)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012.0,2019-03-19,138.084633,10.0,연립다세대
1,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012.0,2019-03-19,138.084633,10.0,연립다세대
2,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012.0,2019-03-19,138.084633,10.0,연립다세대
3,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,2,2012.0,2019-03-19,138.084633,10.0,연립다세대
4,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012.0,2019-03-19,138.084633,10.0,연립다세대


In [11]:
'''land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()'''

"land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')\nprint(land_specs_df.shape)\nland_specs_df.head()"

In [12]:
'''last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)
print(last_df.shape)
last_df.head()'''

"last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)\nprint(last_df.shape)\nlast_df.head()"

In [13]:
'''house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)
print(house_merge_df.shape)'''

"house_merge_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)\nprint(house_merge_df.shape)"

In [14]:
house_df['일괄계약'] = 'N'

In [15]:
#house_df = house_df.merge(last_df, on=['지번주소']).reset_index(drop=True)

In [16]:
'''print(house_df.shape)
house_df.head()'''

'print(house_df.shape)\nhouse_df.head()'

In [17]:
def identify_bulk_contract(df):
    df = df.copy()
    
    for addr in tqdm.tqdm_notebook(df['지번주소'].unique()):
        addr_df = df[df['지번주소'] == addr]
        
        for btyear in addr_df['건축년도'].unique():
            btyear_df = addr_df[addr_df['건축년도'] == btyear]
            
            if (btyear_df.shape[0] >= 10) & (btyear_df['계약날짜'].nunique() == 1):
                df.loc[btyear_df.index, '일괄계약'] = 'Y'

    return df

In [18]:
house_df['계약날짜'].describe()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                   89400
unique                   1513
top       2018-06-27 00:00:00
freq                      587
first     2018-01-01 00:00:00
last      2022-02-23 00:00:00
Name: 계약날짜, dtype: object

In [19]:
house_df = identify_bulk_contract(house_df)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/2625 [00:00<?, ?it/s]

In [20]:
bulk_df = house_df[house_df['일괄계약'] == 'Y']
print(bulk_df.shape)
bulk_df.head()

(716, 15)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜,단가(만원/㎡),건물연식,부동산유형,일괄계약
0,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012.0,2019-03-19,138.084633,10.0,연립다세대,Y
1,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012.0,2019-03-19,138.084633,10.0,연립다세대,Y
2,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,4,2012.0,2019-03-19,138.084633,10.0,연립다세대,Y
3,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,2,2012.0,2019-03-19,138.084633,10.0,연립다세대,Y
4,강원도 강릉시 강동면 산성우리 219-0,오이동길 45-4,강원도 강릉시 강동면 산성우리,219,0,정동진우리하우스텔1,22.45,3100,3,2012.0,2019-03-19,138.084633,10.0,연립다세대,Y


In [21]:
bulk_df['지번주소'].nunique()

26

In [22]:
%%time
bulk_df.to_excel('./공공주택찾기/강원도/일괄계약전체_20180101_20220224.xlsx', index=False)

Wall time: 324 ms
